# 1. Connect to Azure Machine Learning Workspace

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

In [2]:
# import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()
# connect to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="efc5a0e2-7106-4a45-88f4-c9359bc36a6f",
    resource_group_name="eydatabook_rg",
    workspace_name="mlw-ranjitha-from-spyder",
)

# 2. Configure and run the Command

## 2.1 Configure the Command

In [3]:
from azure.ai.ml import command, Input

# define the command
command_job = command(
    code="./src",
    command="python main.py --iris-csv ${{inputs.iris_csv}} --learning-rate ${{inputs.learning_rate}} --boosting ${{inputs.boosting}}",
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    inputs={
        "iris_csv": Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/iris.csv",
        ),
        "learning_rate": 0.9,
        "boosting": "gbdt",
    },
    compute="ci-aml-ranjitha",
)

## 2.2 Run the Command

In [4]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Uploading src (0.01 MBs): 100%|██████████| 6156/6156 [00:00<00:00, 57844.17it/s]




'https://ml.azure.com/runs/goofy_apricot_2v8sbb5w6y?wsid=/subscriptions/efc5a0e2-7106-4a45-88f4-c9359bc36a6f/resourcegroups/eydatabook_rg/workspaces/mlw-ranjitha-from-spyder&tid=7c55576f-e738-4097-8c71-053078eadf3f'

# 3.  Run a sweep on this command

## 3.1 Modify or define inputs to sweep

In [5]:
command_job_for_sweep = command_job(
    learning_rate=Uniform(min_value=0.01, max_value=0.9),
    boosting=Choice(values=["gbdt", "dart"]),
)

## 3.2 Define the sweep function 

In [6]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="test-multi_logloss",
    goal="Minimize",
)

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=20, max_concurrent_trials=10, timeout=7200)

## 3.3 Run the SweepJob

In [7]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)
# get a URL for the status of the job
returned_sweep_job.studio_url

'https://ml.azure.com/runs/bold_nutmeg_6ddctxw82k?wsid=/subscriptions/efc5a0e2-7106-4a45-88f4-c9359bc36a6f/resourcegroups/eydatabook_rg/workspaces/mlw-ranjitha-from-spyder&tid=7c55576f-e738-4097-8c71-053078eadf3f'

## Find the best trial job

In [ ]:
ml_client.jobs.download(returned_sweep_job.name, output_name="model")